In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

Application of the TF-DF tutorial available at: https://www.tensorflow.org/decision_forests/tutorials/intermediate_colab

it's highly recommended to set iper-parameters,  apply cv folds..

In [ ]:
!pip install tensorflow_decision_forests

In [ ]:
!pip install wurlitzer

In [ ]:
import os
import numpy as np
import pandas as pd
import tensorflow as tf
import math
import tensorflow_decision_forests as tfdf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import RobustScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OrdinalEncoder
tf.random.set_seed(22)
import keras
import matplotlib.pyplot as plt

try:
    from wurlitzer import sys_pipes
except:
    from colabtools.googlelog import CaptureLog as sys_pipes

In [ ]:
train= pd.read_csv('/kaggle/input/tabular-playground-series-feb-2022/train.csv', sep=',', index_col='row_id')
train.head()

In [ ]:
test= pd.read_csv('/kaggle/input/tabular-playground-series-feb-2022/test.csv', sep=',', index_col='row_id')
test.head()

In [ ]:
train = train.drop_duplicates(keep='first')

In [ ]:
train['target'].value_counts(normalize=True).plot(kind='bar', title='Target Distribution')

In [ ]:
le = LabelEncoder()
target = pd.DataFrame(le.fit_transform(train['target']), index=train.index)

In [ ]:
ds_train = tfdf.keras.pd_dataframe_to_tf_dataset(train, label='target')
test_ds = tfdf.keras.pd_dataframe_to_tf_dataset(test)

In [ ]:
train=0
test=0

## TFDF Random Forest

In [ ]:
# Creating the Random Forest model with parameter adjusted
rf_model = tfdf.keras.RandomForestModel(try_resume_training=False,winner_take_all=False, categorical_algorithm='RANDOM' , bootstrap_training_dataset=False, sampling_with_replacement=False, num_trees=1700, bootstrap_size_ratio=0.01 ,max_depth=300, verbose=1, temp_directory='model_rf')

In [ ]:
rf_model.compile(metrics=[tf.keras.metrics.Accuracy()])
with sys_pipes():
    rf_model.fit(x=ds_train)

In [ ]:
p2 = rf_model.predict(test_ds)

# Submission

In [ ]:
sub= pd.read_csv('/kaggle/input/tabular-playground-series-feb-2022/sample_submission.csv', sep=',', index_col='row_id')
sub.head()

In [ ]:
predictions= pd.DataFrame(p2, index=test.index)
preds= predictions.idxmax(axis=1)
preds = pd.DataFrame(preds, index=test.index, columns=['target'])
preds['target']=le.inverse_transform(preds['target'])

In [ ]:
sub['target']=preds['target']

In [ ]:
sub = sub.reset_index()
sub.to_csv('submission.csv',index=False)